# DEMO

In [1]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import csv
from scipy import io
import pickle
from IPython.display import Audio, display

In [2]:
def showAudio(info):
    display(Audio(info['previews']['preview-lq-mp3']))

def load_pickle(fname):
    f = open(fname, 'rb')
    out = pickle.load(f)
    f.close()
    return out

In [3]:
train_tags  = load_pickle('data/train_tags.pickle3')
train_mfccs = load_pickle('data/train_mfccs.pickle3')
train_info  = load_pickle('data/train_info.pickle3')

test_mfccs = load_pickle('data/test_mfccs.pickle3')
test_info  = load_pickle('data/test_info.pickle3')

In [4]:
# compute delta MFCCs
def compute_delta_mfccs(mfccs):
    dmfccs = []
    for m in mfccs:
        tmp = m[1:] - m[0:-1]
        dm = hstack((m[0:-1], tmp))
        dmfccs.append(dm)
    return dmfccs

In [5]:
train_dmfccs = compute_delta_mfccs(train_mfccs)
test_dmfccs  = compute_delta_mfccs(test_mfccs)

In [6]:
tagnames, tagnames_counts = unique(concatenate(train_tags), return_counts=True)
for a,b in zip(tagnames, tagnames_counts):
    print("{}: {}".format(a, b))

acoust: 100
analog: 100
bass: 160
beat: 128
drum: 371
effect: 141
electron: 194
field: 110
glitch: 110
guitar: 130
hit: 110
loop: 237
machin: 100
metal: 117
nois: 199
percuss: 285
record: 192
space: 125
synth: 220
synthes: 136
vocal: 120
voic: 167


In [7]:
# convert list of tags into binary class labels
def tags2class(tags, tagnames):
    b = zeros(shape=(len(tags), len(tagnames)))
    for i,t in enumerate(tags):
        for j,n in enumerate(tagnames):
            if n in t:
                b[i,j] = 1
    return b

In [8]:
# train_classes[i,j] = absence/presence of the j-th tag in the i-th sound
train_classes = tags2class(train_tags, tagnames)

In [9]:
# double check we did this correctly...
# it should be the same as the tag counts above
sum(train_classes,axis=0)

array([100., 100., 160., 128., 371., 141., 194., 110., 110., 130., 110.,
       237., 100., 117., 199., 285., 192., 125., 220., 136., 120., 167.])

In [10]:
import csv

def write_csv_kaggle_tags(fname, tagnames, Yscores):
    # header
    tmp = [['Id']]
    for t in tagnames:
        tmp[0].append(t)    
    
    # add ID numbers for each Y, and usage if necessary
    for i in range(len(Yscores)):
        tmp2 = [(i+1)]
        for t in range(len(tagnames)):
            tmp2.append(Yscores[i,t])
        
        tmp.append(tmp2)
        
    # write CSV file
    f = open(fname, 'w')
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

## YOUR CODE and DOCUMENTATION HERE

In [11]:
num_classes = len(tagnames)
num_train = len(train_mfccs)
num_test = len(test_mfccs)
pos_cnt = train_classes.sum(0)

print(f'num_classes: {num_classes}')
print(f'number of training samples: {num_train}')
print(f'number of testing samples: {num_test}')
print(f'number of positives in each class:\n {pos_cnt}')


print(f'train_mfccs[0].shape: {train_mfccs[0].shape}')
print(f'train_classes.shape: {train_classes.shape}')

num_classes: 22
number of training samples: 1788
number of testing samples: 262
number of positives in each class:
 [100. 100. 160. 128. 371. 141. 194. 110. 110. 130. 110. 237. 100. 117.
 199. 285. 192. 125. 220. 136. 120. 167.]
train_mfccs[0].shape: (345, 13)
train_classes.shape: (1788, 22)


In [12]:
import torch
from torch.utils.data import DataLoader, random_split
import numpy as np
import logging
import os

# dataset
from extra.dataset import MFCCDataset, MyPadCollate

# train utils
from extra.train_utils import train, evaluate, AUROC, \
                              grid_search, load_model

LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

# https://pytorch.org/docs/stable/notes/randomness.html
SEED = 10086
torch.manual_seed(SEED)
np.random.seed(SEED)
# https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936/3
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

### prepare dataset and data loader

In [13]:
mfcc_dataset = MFCCDataset(train_mfccs, train_classes)
allloader = DataLoader(mfcc_dataset,
                             batch_size=64,
                             collate_fn=MyPadCollate(batch_first=True))

In [14]:
train_val_split_ratio = 0.9
train_num = int(train_val_split_ratio * len(mfcc_dataset))
valid_num = len(mfcc_dataset) - train_num
mfcc_train, mfcc_valid = random_split(mfcc_dataset,
                                      (train_num, valid_num))

print(len(mfcc_train))
print(len(mfcc_valid))

1609
179


In [15]:
trainloader = DataLoader(mfcc_train,
                         batch_size=64,
                         collate_fn=MyPadCollate(batch_first=True))

validloader = DataLoader(mfcc_valid,
                       batch_size=64,
                       collate_fn=MyPadCollate(batch_first=True))

### confirm scores are correct

In [16]:
best_save_path = 'ckpt_backup/gru_mcauc_0.864.pt'
model = load_model(best_save_path)

allloader = DataLoader(mfcc_dataset,
                       batch_size=128,
                       collate_fn=MyPadCollate(True)
                      )

pred_all, (mc_auc_all, auc_all) = evaluate(model, allloader,
                                           AUROC(len(tagnames)))
print(f'mc_auc_all: {mc_auc_all:.4f}, auc_all: {auc_all:.4f}')

pred_valid, (mc_auc_valid, auc_valid) = evaluate(model, validloader,
                                                AUROC(len(tagnames)))
print(f'mc_auc_valid: {mc_auc_valid:.4f}, auc_valid: {auc_valid:.4f}')

mc_auc_all: 0.9740, auc_all: 0.9741
mc_auc_valid: 0.9658, auc_valid: 0.9673


### write submission

In [17]:
n = len(tagnames)
pseudo_test_label = np.random.randint(2, size=(len(test_mfccs), n))

test_dataset = MFCCDataset(test_mfccs, pseudo_test_label)
test_loader = DataLoader(test_dataset,
                         batch_size=128,
                         shuffle=False,
                         collate_fn=MyPadCollate(True))

test_preds, _ = evaluate(model, test_loader,
                         AUROC(len(tagnames)))

In [18]:
save_name = os.path.basename(best_save_path)[:-2]+'csv'
write_csv_kaggle_tags(save_name, tagnames, test_preds)